Resizing the REAL and FAKE images to 256x256

In [34]:
import os
import zipfile
from PIL import Image
import numpy as np
import cv2

real_folder = '/Users/siddharthiyer/Desktop/detect ai artwork - resizing images for efficiency/ai vs real art/REAL'
fake_folder = '/Users/siddharthiyer/Desktop/detect ai artwork - resizing images for efficiency/ai vs real art/FAKE'

def gauss_kernel(size, sigma):
    size = int(size) // 2
    x, y = np.mgrid[-size:size+1, -size:size+1]
    g = np.exp(-((x**2 + y**2) / (2.0*sigma**2))) * (1 / (2.0 * np.pi * sigma**2))
    g /= g.sum() # Normalize
    return g

def subsample(image_array):
    R = image_array[:, :, 0]
    G = image_array[:, :, 1]
    B = image_array[:, :, 2]

    R_sub = R[::2, :]
    G_sub = G[::2, :]
    B_sub = B[::2, :]
    R_sub = R_sub[:, ::2]
    G_sub = G_sub[:, ::2]
    B_sub = B_sub[:, ::2]

    subsampled_array = np.stack([R_sub, G_sub, B_sub], axis=-1)

    return subsampled_array

In [36]:
# Resize images

# Images begin as 512x512. For every image, blur with gauss kernel and convolution code, then subsample to 256x256
# Save the new images in a new folder

blur_kernel = gauss_kernel(5, 1)

real_resized = []
fake_resized = []

for filename in os.listdir(real_folder):
    image = Image.open(real_folder + '/' + filename)
    image = np.array(image)
    image_resized = cv2.filter2D(image, -1, blur_kernel)
    image_resized = subsample(image_resized)
    real_resized.append(image_resized)

for filename in os.listdir(fake_folder):
    image = Image.open(real_folder + '/' + filename)
    image = np.array(image)
    image_resized = cv2.filter2D(image, -1, blur_kernel)
    image_resized = subsample(image_resized)
    fake_resized.append(image_resized)

# Create folders for real and fake resized
# os.mkdir(real_folder + '_resized')
# os.mkdir(fake_folder + '_resized')

# Save images
for i, image in enumerate(real_resized):
    cv2.imwrite(real_folder + '_resized/' + str(i) + '.jpg', image)

for i, image in enumerate(fake_resized):
    cv2.imwrite(fake_folder + '_resized/' + str(i) + '.jpg', image)


Simple Convolutional Neural Network Architecture to Predict Real vs. AI Images


In [ ]:
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Set the paths to the resized image folders
real_folder = '/content/REAL_resized/'
fake_folder = '/content/FAKE_resized/'

# Set the image size and batch size
img_size = (256, 256)
batch_size = 32

# Create data generators for training and validation
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    directory='/content/',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    directory='/content/',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'
)

# Build the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
epochs = 10
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    epochs=epochs
)

Found 5456 images belonging to 7 classes.
Found 1364 images belonging to 7 classes.
Epoch 1/10
170/170 [==============================] - 762s 4s/step - loss: -19611795456.0000 - accuracy: 0.0000e+00 - val_loss: -146565365760.0000 - val_accuracy: 0.0000e+00
Epoch 2/10
 20/170 [==>...........................] - ETA: 9:50 - loss: -202139123712.0000 - accuracy: 0.0000e+00

KeyboardInterrupt: 